<a href="https://colab.research.google.com/github/ranjeni7/Rubber_properties_prediction/blob/main/Rubber_properties_prediction_using_tabnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to upload a file manually

Saving synthetic_dataset.csv to synthetic_dataset.csv


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = "/content/drive/My Drive/synthetic_dataset.csv"

In [5]:
!pip install pytorch-tabnet pandas scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [6]:
import pandas as pd

# Load dataset
file_path = "synthetic_dataset.csv"  # Change if needed
df = pd.read_csv(file_path)

# Display first few rows
df.head()

,100% Modulus (MPa),Tensile Strength (MPa),Elongation at Break (%),Volume Fraction of Rubber,Dose (kGy),Sensitizer (phr),Filler (phr),Antioxidant (phr),Sulfur (phr)
0,0.660716,4.431898,771.732720,0.332959,72.691521,1.393283,15.778081,0.0,1.318090
1,1.018914,4.868648,791.700458,0.153071,87.797855,0.149520,3.605957,0.0,0.656678
2,1.057123,6.968434,720.110470,0.366903,134.485528,2.859961,10.637336,0.0,0.889776
3,1.079370,9.486072,1006.076872,0.185073,138.900807,2.388221,18.143664,0.0,0.775947
4,1.063454,3.044374,348.760100,0.368397,140.110146,1.325135,38.459965,0.0,0.855385


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define input (X) and output (Y) variables correctly
X = df[["100% Modulus (MPa)", "Tensile Strength (MPa)", "Elongation at Break (%)", "Volume Fraction of Rubber"]]  # Inputs
Y = df[["Dose (kGy)", "Sensitizer (phr)", "Filler (phr)", "Antioxidant (phr)", "Sulfur (phr)"]]  # Outputs

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize only the input data (output should not be scaled)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np

# Convert data to NumPy arrays (TabNet requires this format)
X_train_np, X_test_np = np.array(X_train_scaled), np.array(X_test)
Y_train_np, Y_test_np = np.array(Y_train), np.array(Y_test)

# Initialize TabNet model
tabnet_model = TabNetRegressor()

# Train model
tabnet_model.fit(
    X_train_np, Y_train_np,
    eval_set=[(X_test_np, Y_test_np)],
    max_epochs=100, patience=10,
    batch_size=128, virtual_batch_size=32,
    num_workers=0,
)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 6699.67708| val_0_mse: 6013.63453|  0:00:01s
epoch 1  | loss: 6648.25667| val_0_mse: 4936.53383|  0:00:01s
epoch 2  | loss: 6590.13493| val_0_mse: 4373.50342|  0:00:01s
epoch 3  | loss: 6506.66227| val_0_mse: 3682.9679|  0:00:01s
epoch 4  | loss: 6420.05973| val_0_mse: 3377.0791|  0:00:01s
epoch 5  | loss: 6132.06527| val_0_mse: 3317.29803|  0:00:01s
epoch 6  | loss: 6088.26904| val_0_mse: 3039.69977|  0:00:02s
epoch 7  | loss: 5923.67448| val_0_mse: 2905.19697|  0:00:02s
epoch 8  | loss: 5803.53841| val_0_mse: 40469.6141|  0:00:02s
epoch 9  | loss: 5629.96501| val_0_mse: 360879.22773|  0:00:02s
epoch 10 | loss: 5444.16309| val_0_mse: 2298800.40173|  0:00:02s
epoch 11 | loss: 5278.73503| val_0_mse: 3107871.85934|  0:00:02s
epoch 12 | loss: 5060.60042| val_0_mse: 1366745.71968|  0:00:02s
epoch 13 | loss: 4876.01416| val_0_mse: 1184243.44451|  0:00:02s
epoch 14 | loss: 4655.64844| val_0_mse: 1639023.10894|  0:00:02s
epoch 15 | loss: 4440.66732| val_0_mse: 1340530.51321| 

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [9]:
# Get predictions
Y_pred = tabnet_model.predict(X_test_np)

# Evaluate performance using Mean Absolute Error (MAE)
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test, Y_pred)
print(f"Mean Absolute Error: {mae:.4f}")

Mean Absolute Error: 35.3379


In [10]:
# Example new input data (single row from X_test)
sample_input = X_test_np[0].reshape(1, -1)

# Predict using trained model
predicted_output = tabnet_model.predict(sample_input)
print("Predicted Values:", predicted_output)

Predicted Values: [[160.08516   16.154638  12.858062  37.323708 -40.31997 ]]


In [13]:
# Function to predict from manual input
def predict_custom_input(modulus, tensile_strength, elongation, volume_fraction):
    import numpy as np

    # Convert input into a NumPy array (as model expects)
    manual_input = np.array([[modulus, tensile_strength, elongation, volume_fraction]])

    # Scale input (must use the same scaler used for training)
    manual_input_scaled = scaler.transform(manual_input)

    # Predict using trained model
    predicted_output = tabnet_model.predict(manual_input_scaled)

    # Display results
    print("\n🔮 Predicted Values 🔮")
    print(f"Dose (kGy): {predicted_output[0][0]:.2f}")
    print(f"Sensitizer (phr): {predicted_output[0][1]:.2f}")
    print(f"Filler (phr): {predicted_output[0][2]:.2f}")
    print(f"Antioxidant (phr): {predicted_output[0][3]:.2f}")
    print(f"Sulfur (phr): {predicted_output[0][4]:.2f}")

# Example: Manually enter values
modulus_value = float(input("Enter 100% Modulus (MPa): "))
tensile_strength_value = float(input("Enter Tensile Strength (MPa): "))
elongation_value = float(input("Enter Elongation at Break (%): "))
volume_fraction_value = float(input("Enter Volume Fraction of Rubber: "))

# Predict output
predict_custom_input(modulus_value, tensile_strength_value, elongation_value, volume_fraction_value)

Enter 100% Modulus (MPa): 0.98
Enter Tensile Strength (MPa): 2.9
Enter Elongation at Break (%): 519
Enter Volume Fraction of Rubber: 0.149

🔮 Predicted Values 🔮
Dose (kGy): 8.02
Sensitizer (phr): 3.69
Filler (phr): 2.51
Antioxidant (phr): -3.60
Sulfur (phr): 2.18


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
tabnet_model.fit(
    X_train_scaled, Y_train.values,  # Convert Y_train to NumPy array
    eval_set=[(X_test_scaled, Y_test.values)],  # Convert Y_test to NumPy array
    eval_name=["valid"],
    eval_metric=["rmse"],
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
)

epoch 0  | loss: 6905.29053| valid_rmse: 80.24289|  0:00:00s
epoch 1  | loss: 6930.6499| valid_rmse: 79.81506|  0:00:00s
epoch 2  | loss: 6599.26758| valid_rmse: 79.58363|  0:00:00s
epoch 3  | loss: 6831.8457| valid_rmse: 79.31119|  0:00:00s
epoch 4  | loss: 6690.12158| valid_rmse: 78.99064|  0:00:00s
epoch 5  | loss: 6564.48486| valid_rmse: 78.7232 |  0:00:00s
epoch 6  | loss: 6600.87695| valid_rmse: 78.45745|  0:00:00s
epoch 7  | loss: 6576.33789| valid_rmse: 78.21156|  0:00:00s
epoch 8  | loss: 6494.75586| valid_rmse: 78.00058|  0:00:00s
epoch 9  | loss: 6277.48242| valid_rmse: 77.55622|  0:00:00s
epoch 10 | loss: 6095.65234| valid_rmse: 77.06142|  0:00:00s
epoch 11 | loss: 6279.25488| valid_rmse: 76.72744|  0:00:00s
epoch 12 | loss: 6257.26709| valid_rmse: 76.2651 |  0:00:00s
epoch 13 | loss: 6601.82617| valid_rmse: 75.85646|  0:00:00s
epoch 14 | loss: 6527.38818| valid_rmse: 75.50588|  0:00:00s
epoch 15 | loss: 6240.43213| valid_rmse: 75.21996|  0:00:00s
epoch 16 | loss: 6291.9755

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predict on the test set
Y_pred = tabnet_model.predict(X_test_scaled)

# Compute performance metrics
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

# Print results
print(f"📉 Mean Squared Error (MSE): {mse:.4f}")
print(f"📊 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"📌 Mean Absolute Error (MAE): {mae:.4f}")
print(f"🔹 R² Score: {r2:.4f}")

📉 Mean Squared Error (MSE): 1503.3484
📊 Root Mean Squared Error (RMSE): 38.7730
📌 Mean Absolute Error (MAE): 17.0560
🔹 R² Score: -1.9336
